In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import AnalysisFunctions as afuns
from functools import reduce
import os
import networkx as nx
import seaborn as sns
import scipy.stats as stats
import plotly.express as px

In [83]:
# pip install --upgrade networkx

In [84]:
root_path = r'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')

In [85]:
def get_bdata_filename(exp_path):
    bdata_filename = [filename for filename in os.listdir(exp_path+sep+r'with food\blob analysis normalized by white paper') 
                        if filename.startswith(r'bdata')][0]
    return bdata_filename

In [86]:
%matplotlib qt

In [87]:
def get_experiment_data(exp_num, exp_details, root_path):
    exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)
    #bdata_filename = get_bdata_filename(exp_path)
    tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
    ants = pd.read_csv(exp_path + sep + r'ants_list.csv')
    foragers=[ant['ant_id'] for idx, ant in ants.iterrows() if ant['is_forager']]
    
    return exp_idx, PC_ratios_dict, exp_path, tdata, ants, foragers

In [88]:
def make_givers_df(tdata, foragers):
    givers=tdata.groupby('giver')[['transferred_yellow','transferred_red']].apply(sum)
    givers['is_forager']=givers.index.isin(foragers)
    givers['num_of_interactions']=tdata.groupby('giver').size()
    return givers

In [89]:
def make_receivers_df(tdata, foragers):
    receivers=tdata.groupby('receiver')[['transferred_yellow','transferred_red']].apply(sum)
    receivers['is_forager']=receivers.index.isin(foragers)
    receivers['num_of_interactions']=tdata.groupby('receiver').size()
    return receivers

In [90]:
def scatter_total_transferred(ax, givers, max_value, exp_num):
    g=sns.scatterplot(data=givers,x='transferred_yellow', y='transferred_red',  hue='is_forager',legend=True,ax=ax)
    g.legend_.set_title(None)
    for t, l in zip(g.legend_.texts, ['Non-forager','Forager']):
        t.set_text(l)
    g.set_xlabel('Total transferred yellow [ul]')
    g.set_ylabel('Total transferred red [ul]')
    g.set_aspect('equal')
    g.set(xlim=(0,max_value+10), ylim=(0,max_value+10))
    g.set_title(f'Experiment {exp_num}')

In [102]:
def histogram_with_gamma_fit(ax, data, color, exp_num, bins=np.arange(0,300,15), floc=-0.01):
    
    ax.hist(data,color=color,bins=bins,density=True)
    ax.set_xlabel('Total transferred [ul]')
    ax.set_ylabel('pdf')
    ax.set_title(f'Experiment {exp_num}')

    #data=givers.sum(axis=1).values
    k,loc1,theta=stats.gamma.fit(data,floc=floc)
    (k,loc1,theta)
    x=np.linspace(3,300,100)
    gamma = stats.gamma.pdf(x, k, loc1, theta)
    ax.plot(x,gamma,'r')
    ax.set_title(f'Exp. {exp_num} | '+
                      f'k={k:.2f},'+r'$\theta$'+f'={10/theta:.2f}')

In [103]:
exp_idx, PC_ratios_dict, exp_path, tdata, ants, foragers = get_experiment_data(20, exp_details, root_path)
givers = make_givers_df(tdata, foragers)
givers.head()

,transferred_yellow,transferred_red,is_forager,num_of_interactions
giver,,,,
-5,6.516212,7.269922,False,15
1,2.131253,35.095044,False,18
5,0.118545,23.707750,False,21
30,1.411550,8.638616,False,11
33,4.280116,3.252054,False,6


In [104]:
receivers = make_receivers_df(tdata, foragers)
receivers.rename(columns={'transferred_yellow':'received_yellow', 'transferred_red':'received_red', 
                          'num_of_interactions':'num_of_receiver_interactions'},inplace=True)
receivers.index.rename('ant',inplace=True)
receivers.head()

,received_yellow,received_red,is_forager,num_of_receiver_interactions
ant,,,,
-5,7.169588,19.795392,False,9
1,2.617595,33.729475,False,22
5,3.589878,34.261890,False,25
30,0.663876,29.034725,False,12
33,9.942672,29.373466,False,10


In [105]:
givers_new = givers.rename(columns={'transferred_yellow':'gived_yellow', 'transferred_red':'gived_red', 
                          'num_of_interactions':'num_of_giving_interactions'})
givers_new.index.rename('ant',inplace=True)
df=givers_new.merge(receivers, on='ant')
df=df.reset_index()
df.head()

,ant,gived_yellow,gived_red,is_forager_x,num_of_giving_interactions,received_yellow,received_red,is_forager_y,num_of_receiver_interactions
0,-5,6.516212,7.269922,False,15,7.169588,19.795392,False,9
1,1,2.131253,35.095044,False,18,2.617595,33.729475,False,22
2,5,0.118545,23.707750,False,21,3.589878,34.261890,False,25
3,30,1.411550,8.638616,False,11,0.663876,29.034725,False,12
4,33,4.280116,3.252054,False,6,9.942672,29.373466,False,10


In [106]:
fig= px.scatter(df, x='gived_yellow', color='is_forager_x', y='received_yellow', hover_data=['ant'])
fig.show()

# fig,ax=plt.subplots(1,1)
# g=sns.scatterplot(data=df,x='gived_yellow', y='received_yellow',  hue='is_forager_x',legend=True, ax=ax)
# g.legend_.set_title(None)
# for t, l in zip(g.legend_.texts, ['Non-forager','Forager']):
#     t.set_text(l)
# g.set_xlabel('Total given yellow [ul]')
# g.set_ylabel('Total received yellow [ul]')
# g.set_aspect('equal')
#g.set(xlim=(0,max_value+10), ylim=(0,max_value+10))
#g.set_title(f'Experiment {exp_num}')
fig2= px.scatter(df, x='gived_red', color='is_forager_x', y='received_red', hover_data=['ant'])
fig2.show()


In [107]:
plt.figure()
givers.num_of_interactions.hist(bins=np.arange(0,100,5))

In [108]:
givers['transferred_yellow']+givers['transferred_red']

giver
-5        13.786134
 1        37.226297
 5        23.826296
 30       10.050165
 33        7.532170
 37        8.852449
 38       30.535509
 42        0.302802
 43       32.902357
 44       17.807342
 46        6.467433
 47      167.852156
 56       35.872682
 61      170.657091
 64        4.344161
 150      61.517101
 170      55.806188
 208       6.447303
 209      75.190237
 225      34.409008
 236      66.708557
 386      86.567317
 389      20.280291
 424     103.966438
 525       5.547822
 627       9.816927
 673      63.555287
 678      12.788782
 700      16.363743
 740      35.402765
 910       7.782686
 941      14.371488
 1004     19.132407
 1055     14.405651
 1063      4.138149
 1068     51.084770
 1079     23.121269
 1117     26.347670
 1124    239.379534
 1230     32.850962
 1235    263.169311
 1292     57.773410
 1299     70.863464
 1321      7.061008
 1329      7.211135
 1354     17.370742
 1355     35.852146
 1384      0.251196
 1408      2.158459
dtype: float64

In [110]:
experiments = [11,18,19,20,16,17,21,0,22,23,24]
colors = ['blue']*4
colors.extend(['purple']*4)
colors.extend(['green']*3)

fig1, axes1 = plt.subplots(3,4)
fig2, axes2 = plt.subplots(3,4,sharex=True,sharey=False)
fig3, axes3 = plt.subplots(3,4,sharex=True,sharey=False)
fig4, axes4 = plt.subplots(3,4,sharex=True,sharey=False)
fig5, axes5 = plt.subplots(3,4,sharex=True,sharey=False)
fig2.suptitle('total transferred')
fig3.suptitle('yellow')
fig4.suptitle('red')
fig5.suptitle('number of interactions')
for exp_num, color, ax1, ax2, ax3, ax4, ax5 in zip(experiments, colors, axes1.ravel(), axes2.ravel(),
                                                   axes3.ravel(), axes4.ravel(), axes5.ravel()):
    if exp_num>0:
        exp_idx, PC_ratios_dict, exp_path, tdata, ants, foragers = get_experiment_data(exp_num, exp_details, root_path)
        givers = make_givers_df(tdata, foragers)
        
        max_value = max(max(givers['transferred_yellow']),max(givers['transferred_red']))
        
        scatter_total_transferred(ax1, givers, max_value, exp_num)
        
        histogram_with_gamma_fit(ax2, givers['transferred_yellow']+givers['transferred_red'], color, exp_num)
        histogram_with_gamma_fit(ax3, givers['transferred_yellow'], color, exp_num)
        histogram_with_gamma_fit(ax4, givers['transferred_red'], color, exp_num)
        histogram_with_gamma_fit(ax5, givers['num_of_interactions'], color, exp_num, bins=np.arange(0,100,5))
        ax5.set_xlabel('Number of interactions')
        mean_num_interactions = givers['num_of_interactions'].mean()
        plt.text(150,0.15,f"mean {mean_num_interactions: .2f}")

fig1.delaxes(axes1.ravel()[-1])
fig2.delaxes(axes2.ravel()[-1])
fig3.delaxes(axes3.ravel()[-1])
fig4.delaxes(axes4.ravel()[-1])
fig5.delaxes(axes5.ravel()[-1])
fig1.delaxes(axes1.ravel()[7])
fig2.delaxes(axes2.ravel()[7])
fig3.delaxes(axes3.ravel()[7])
fig4.delaxes(axes4.ravel()[7])
fig5.delaxes(axes5.ravel()[7])

In [100]:

experiments = [11,18,19,20,16,17,21,0,22,23,24]
colors = ['blue']*4
colors.extend(['purple']*4)
colors.extend(['green']*3)

fig1, axes1 = plt.subplots(3,4)
fig2, axes2 = plt.subplots(3,4,sharex=True,sharey=False)
for exp_num, color,ax1,ax2 in zip(experiments, colors,axes1.ravel(),axes2.ravel()):
    if exp_num>0:
        exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)
        bdata_filename = get_bdata_filename(exp_path)

        # bdata = pd.read_csv(exp_path + sep + r'with food\blob analysis normalized by white paper' + sep + bdata_filename, index_col=0)
        # fdata = pd.read_csv(exp_path + sep + r'forager_table_with_feeding_sizes_ul_transparency_corrected.csv', index_col=[0])
        tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
        # crops = pd.read_csv(exp_path + sep + r'clean_crops_transparency_corrected.csv', header=[0,1], index_col=[0]).swaplevel(axis=1)
        ants = pd.read_csv(exp_path + sep + r'ants_list.csv')
        # conversion_factors = pd.read_csv(exp_path + sep + r'conversion_factors_by_weight_and_feeding_sum.csv')
        # transparency = pd.read_csv(exp_path + sep + r'transparency_table.csv')

        foragers=[ant['ant_id'] for idx, ant in ants.iterrows() if ant['is_forager']]

        givers=tdata.groupby('giver')[['transferred_yellow','transferred_red']].apply(sum)
        givers['is_forager']=givers.index.isin(foragers)

        max_value = max(max(givers['transferred_yellow']),max(givers['transferred_red']))


        #plt.figure()
        g=sns.scatterplot(data=givers,x='transferred_yellow', y='transferred_red',  hue='is_forager',legend=True,ax=ax1)
        g.legend_.set_title(None)
        for t, l in zip(g.legend_.texts, ['Non-forager','Forager']):
            t.set_text(l)
        g.set_xlabel('Total transferred yellow [ul]')
        g.set_ylabel('Total transferred red [ul]')
        g.set_aspect('equal')
        g.set(xlim=(0,max_value+10), ylim=(0,max_value+10))
        g.set_title(f'Experiment {exp_num}')


        ax2.hist(givers.sum(axis=1),color=color,bins=np.arange(0,300,15),density=True)
        ax2.set_xlabel('Total transferred [ul]')
        ax2.set_ylabel('pdf')
        ax2.set_title(f'Experiment {exp_num}')

        data=givers.sum(axis=1).values
        k,loc1,theta=stats.gamma.fit(data,floc=-0.01)
        (k,loc1,theta)
        x=np.linspace(3,300,100)
        gamma = stats.gamma.pdf(x,k, loc1, theta)
        ax2.plot(x,gamma,'r')
        
        
        
    #     (loc2, sigma) = stats.rayleigh.fit(data, floc=0)
    #     rayleigh = stats.rayleigh.pdf(x,loc2,sigma)
    #     ax2.plot(x,rayleigh,'y')
    #     (loc3, a) = stats.maxwell.fit(data, floc=0)
    #     mb = stats.maxwell.pdf(x,loc3,a)
    #     ax2.plot(x,mb,'g')
    #     ax2.set_title(f'k={k:.2f},'+r'$\theta$'+f'={theta:.2f}\n' +
    #                   f'fit mean: {k*theta:.2f}, mean {np.mean(data)}\n'+
    #                  f'sigma={sigma:.2f}\n'+
    #                  f'a={a:.2f}') #Exp. {exp_num} | loc={loc:.2f},
        ax2.set_title(f'Exp. {exp_num} | '+
                      f'k={k:.2f},'+r'$\theta$'+f'={theta:.2f}')#', loc={loc1:.2f}')#\n' +
                       #f'fit mean: {k*theta:.2f}, mean {np.mean(data):.2f}')
    
fig1.delaxes(axes1.ravel()[-1])
fig2.delaxes(axes2.ravel()[-1])
fig1.delaxes(axes1.ravel()[7])
fig2.delaxes(axes2.ravel()[7])

In [ ]:
plt.suptitle('super title')

In [ ]:
np.mean(data)

In [ ]:
np.var(data)

In [ ]:
k*theta**2

In [ ]:
exp_num

In [ ]:
data=givers.sum(axis=1).values
alpha,loc,beta=stats.gamma.fit(data)
(alpha,loc,beta)

In [ ]:
stats.gamma.fit(data,floc=0)

In [ ]:
def poisson_function(k,lamb):
    return stats.poisson.pmf(k, lamb)

In [ ]:
from scipy.optimize import curve_fit

# the bins should be of integer width, because poisson is an integer distribution
bins = np.arange(0,300,15)
entries, bin_edges, patches = plt.hist(data, bins=bins, density=True, label='Data')

# calculate bin centers
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])


def fit_function(k, lamb):
    '''poisson function, parameter lamb is the fit parameter'''
    return stats.poisson.pmf(k, lamb)


# fit with curve_fit
parameters, cov_matrix = curve_fit(fit_function, bin_centers, entries)

# plot poisson-deviation with fitted parameter
x_plot = bin_centers

plt.plot(
    x_plot,
    fit_function(x_plot, *parameters),
    'orange'
)

In [ ]:
stats.maxwell.fit(data, floc=0)

In [ ]:
np.mean(data)

In [ ]:
alpha*beta

In [ ]:
x=np.linspace(0.1,300,20)

In [ ]:
d = stats.gamma.pdf(x,alpha, loc, beta)
plt.plot(x,d)

In [ ]:
x = np.linspace(stats.gamma.ppf(0.01, a),
                gamma.ppf(0.99, a), 100)
ax.plot(x, gamma.pdf(x, a),
       'r-', lw=5, alpha=0.6, label='gamma pdf')

In [ ]:
res = stats.fit(stats.gamma,data)

In [ ]:
import scipy
scipy.__version__

In [ ]:
pip install scipy==1.10.1

In [ ]:
foragers=[ant['ant_id'] for idx, ant in ants.iterrows() if ant['is_forager']]
foragers

In [ ]:
tdata.head()

In [ ]:
aggregated=tdata.groupby(['giver','receiver'])['transferred_yellow'].apply(sum).reset_index()
aggregated

In [ ]:
['blue']*4, ['purple']*3

In [ ]:
yellow_graph = nx.from_pandas_edgelist(aggregated,source='giver', target='receiver', edge_attr='transferred_yellow', create_using=nx.MultiDiGraph) 

In [ ]:
%matplotlib qt
nx.draw_networkx(yellow_graph,pos=nx.spring_layout(yellow_graph,weight='transferred_yellow'))

In [ ]:
neighbors76 = list(nx.all_neighbors(yellow_graph,76))
neighbors76.append(76)
neighbors76

In [ ]:
yellow76 = nx.induced_subgraph(yellow_graph,neighbors76)

In [ ]:
plt.figure()
nx.draw_networkx(yellow76,pos=nx.spring_layout(yellow76,weight='transferred_yellow'))

In [ ]:
givers=tdata.groupby('giver')[['transferred_yellow','transferred_red']].apply(sum)
givers['is_forager']=givers.index.isin(foragers)

In [ ]:
plt.figure()
plt.scatter(givers['transferred_yellow'],givers['transferred_red'],c=givers['is_forager'],label=givers['is_forager'])
plt.xlabel('Total transferred yellow [ul]')
plt.ylabel('Total transferred red [ul]')
plt.legend()

In [ ]:
g=sns.scatterplot(data=givers,x='transferred_yellow', y='transferred_red',  hue='is_forager',legend=True)
g.legend_.set_title(None)
for t, l in zip(g.legend_.texts, ['Non-forager','Forager']):
    t.set_text(l)
g.set_xlabel('Total transferred yellow [ul]')
g.set_ylabel('Total transferred red [ul]')
g.set_aspect('equal')
g.set(xlim=(0,140), ylim=(0,140))

In [ ]:
red_givers=givers.sort_values(by='transferred_red',ascending=False)
yellow_givers=givers.sort_values(by='transferred_yellow',ascending=False)

In [ ]:
yellow_givers[yellow_givers['transferred_yellow']>20]

In [ ]:
red_givers[red_givers['transferred_red']>20]